In [1]:
import assets.helper as b3
import assets.functions as run


In [2]:
import pandas as pd
import numpy as np
import os

import assets.graphs as g
import random
import requests

### QUOTES PRICES

In [ ]:
fund = run.sys_load_pkl(f'{b3.app_folder}/fund')
quotes = run.sys_load_pkl(f'{b3.app_folder}/quotes')


In [ ]:
fund.keys(), fund['BENS INDUSTRIAIS'].columns

In [ ]:
quotes.keys(), quotes['AERIS'].keys()

In [ ]:
df_preplot = {}
start_time = run.time.time()

for i, (setor, df_fund) in enumerate(fund.items()):
    df_fund = preprocess_data(df_fund)
    df_cd_conta = pivot_data(df_fund)

    df_unique = df_fund.drop_duplicates(subset=['DT_REFER', 'PREGAO']).drop(['CD_CONTA', 'DS_CONTA', 'VL_CONTA'], axis=1)
    df_premerged = pd.merge(df_unique, df_cd_conta, on=['DT_REFER', 'PREGAO'])
    df_resampled = resample_data(df_premerged)
    
    df_merged = merge_with_bigdata(df_resampled, bigdata)
    df_merged = df_merged.set_index('Date', drop=True)
    df_merged = df_merged.groupby('PREGAO', group_keys=False).apply(add_metrics)

    # Clean up the dataframe using the cleanup_dataframe function
    df_merged = cleanup_dataframe(df_merged)

    df_merged = add_metrics(df_merged)

    df_preplot[setor] = df_merged

    print(setor, run.sys_remaining_time(start_time, len(fund), i))


In [ ]:
# Define the path to the folder
company_folder = 'company'
folder_path = os.path.join(b3.app_folder, company_folder)

# Check if the folder does not exist
if not os.path.exists(folder_path):
    # Create the folder
    os.makedirs(folder_path)


In [ ]:
start_time = run.time.time()
for i, (setor, df) in enumerate(df_preplot.items()):
    companies = df['PREGAO'].unique()
    for i2, company in enumerate(companies):
        mask = df['PREGAO'] == company
        df_temp = df[mask]
        try:
            df_temp = run.sys_save_pkl(df_temp, f'{b3.app_folder}/{company_folder}/{company}')
        except Exception as e:
            pass
    print(setor, run.sys_remaining_time(start_time, len(df_preplot), i))


In [ ]:
mask = df_preplot['BENS INDUSTRIAIS']['TICKER'] == 'AERI3'
df_preplot['BENS INDUSTRIAIS'][mask]

### DASH

In [ ]:
company = 'AZEVEDO'
df = run.sys_load_pkl(f'{b3.app_folder}/{b3.company_folder}/{company}')
tickers = df['TICKER'].unique()
# df.columns.to_list()

In [ ]:
df['03.11 - Lucro Líquido']
df['14.04.01 - ROE (Resultado por Patrimônio)']
df['03.01 - Receita Bruta']
df['06.01 - Caixa das Operações']





In [ ]:
graphs = g.construct_graphs(df)
graphs.keys()

In [ ]:
import assets.lines
lines = assets.lines.report
groups = set(entry['title'][:2] for entry in lines)
groups

In [ ]:
for g, (group_key, group) in enumerate(graphs.items()):
    status = True if g == 0 else False
    plots = []
    for l, (line_key, line) in enumerate(group.items()):
        for p, (plot_key, plot_info) in enumerate(line.items()):
            for ticker in df['TICKER'].unique():
                df_ticker = df[df['TICKER'] == ticker]
                if not df.empty:
                    print(group[0][0]['info']['title'])
                    print(line[0]['info']['title'])
                    print(plot_info['info']['title'])
                    run.plot_tweak(plot_info, df_ticker).show();

In [ ]:
for ticker in tickers:
    df = df[df['TICKER'] == ticker]
    print(ticker, len(df))

    if not df.empty:
        for g, (group_key, group) in enumerate(merged_entries.items()):
            status = True if g == 0 else False
            plots = []
            for l, (line_key, line) in enumerate(group.items()):
                for p, (plot_key, plot_info) in enumerate(line.items()):
                    print(ticker)
                    run.plot_tweak(plot_info, df).show();

In [ ]:
fund = run.load_database()


In [ ]:
setores = []
for setor, df in fund.items():
    columns=['SETOR', 'SUBSETOR', 'SEGMENTO', 'PREGAO']
    setores.append(df[columns].drop_duplicates())
    print(setor)
sss = pd.concat(setores)
sss = sss.applymap(run.clean_text)

sss = run.sys_save_pkl(sss, f'{b3.app_folder}/sss')

In [ ]:
sss

In [ ]:
mask = sss['SUBSETOR'] == 'MATERIAL DE TRANSPORTE'
sss[mask]

#### Playground


In [ ]:
company_list = run.sys_load_pkl('company_list')
company_list

In [ ]:
listagem_siglas = ['NM', 'N1', 'N2', 'MA', 'M2', 'MB', 'DR1', 'DR2', 'DR3', 'DRE', 'DRN']
listagem_extenso = ['Novo Mercado', 'Nível 1 de Governança Corporativa', 'Nível 2 de Governança Corporativa', 'Bovespa Mais', 'Bovespa Mais Nível 2', 'Organização Tradicional OTC', 'BDR Nível 1', 'BDR Nível 2', 'BDR Nível 3', 'BDR ETF', 'BDR Não Patrocinado']
list_dict = dict(zip(listagem_siglas, listagem_extenso))
list_dict

#### Merge and update


In [ ]:
driver, wait = run.sys_load_browser()
url = 'https://sistemaswebb3-listados.b3.com.br/listedCompaniesPage/search?language=pt-br' 



In [ ]:
b3_cols = b3.cols_b3_companies + b3.col_b3_companies_extra_columns
companies__local = run.sys_read_or_create_dataframe('company', b3_cols).fillna('')
b3_companies_tickers = run.sys_load_pkl('b3_companies_tickers')

b3_companies_tickers = run.get_b3_tickers(driver, wait, url)
for col in b3_cols:
    if col not in b3_companies_tickers.columns:
        b3_companies_tickers[col] = ''
b3_companies_tickers = b3_companies_tickers[b3_cols]
# b3_companies_tickers = sys_save_pkl(b3_companies_tickers, 'b3_companies_tickers')

key_columns = ['ticker', 'company_name']
len(b3_companies_tickers), len(companies__local)

In [ ]:
merged = pd.merge(companies__local, b3_companies_tickers[:], how='outer', on=key_columns, indicator=True)

update_strict = merged[merged['_merge'] == 'right_only'][key_columns] # only companies new in web
update_broad = merged[merged['_merge'] != 'left_only'][key_columns] # all companies from web
updated = update_strict

len(update_broad), len(update_strict)

In [ ]:
key_columns = ['company_name', 'ticker']
b3_cols = b3.cols_b3_companies + b3.col_b3_companies_extra_columns
b3_cols_float = ['Capital Social']
b3_str_cols = [col for col in b3_cols if col not in  b3_cols_float]
col_types = {col: 'float' if col in b3_cols_float else 'str' for col in b3_cols}


In [ ]:
driver.get(b3.url)
# time.sleep(1)

size = len(updated)
# Iterate through the new companies to extract detailed information
new_companies = []
start_time = run.time.time()
for i, (index, row) in enumerate(updated.iterrows()):
    driver.get(b3.url)
    new_company = run.b3_get_company_info(row, driver, wait)
    new_company = pd.DataFrame([new_company], columns=b3_cols).astype(col_types)
    new_company['Capital Social'] = new_company['Capital Social'].replace(np.nan, 0.0)
    new_company = new_company.replace('nan', '')
    new_company['Capital Social'] = pd.to_numeric(new_company['Capital Social'], errors='coerce').astype('float')

    try:
        if not new_company.empty:
            cnpj = new_company['cnpj'][0]
            url = f'https://cnpj.biz/{cnpj}'
            response = requests.get(url, headers={"User-Agent": random.choice(b3.USER_AGENTS)})
            extra = run.b3_get_company_extra_cnpj_info(response)
            extra = pd.DataFrame([extra], columns=b3_cols).fillna('')
            extra = extra.astype(col_types)
        else:
            extra = pd.DataFrame([], columns=b3_cols)

    except Exception as e:
        pass
    extra = extra.reset_index(drop=True)
    new_company = pd.merge(new_company[b3.cols_b3_companies], extra[b3.col_b3_companies_extra_columns], left_on='cnpj', right_on='CNPJ', how='outer').fillna('').reset_index(drop=True)
    new_companies.append(new_company)
    
    print(run.sys_remaining_time(start_time, size, i), row['ticker'], row['company_name'])

    if (size-i-1) % (b3.bin_size/10) == 0:
        temp = pd.concat(new_companies).reset_index(drop=True)
        temp['Capital Social'] = pd.to_numeric(temp['Capital Social'], errors='coerce').astype('float')
        temp['Capital Social'] = temp['Capital Social'].replace(np.nan, 0.0)
        temp = pd.merge(companies__local, temp, on=b3_cols, how='outer', indicator=False).fillna('').drop_duplicates(subset=key_columns, keep='last').reset_index(drop=True)
        temp = sys_save_and_pickle(temp, 'company')


In [ ]:
temp

In [ ]:
for col in b3_cols:
    d1 = new_company[col].dtype
    d2 = extra[col].dtype
    print(col, d1, d2)
    if d1 != d2:
        print(col, d1, d2)
    

In [ ]:
b3_cols

##### CVM GET CODE

In [ ]:
for i in range(0,1000+1):
    url = f'https://sistemaswebb3-listados.b3.com.br/listedCompaniesPage/main/{i}/ABCD/overview?language=pt-br'
    print(url)

##### Final Company Get

In [ ]:
companies__local = run.sys_read_or_create_dataframe('company', b3_cols)


In [ ]:
companies__local.columns

In [ ]:
mask = companies__local['Estado'] == ''
# mask &= companies__local['cnpj'] != ''
companies__local = companies__local[~mask]

In [ ]:
companies__local = run.sys_save_and_pickle(companies__local, 'companies__local')

### Playground

In [ ]:
# Expandindo os DataFrames para terem 5 linhas cada, com categorias nome e fruta em df1
# e categorias de trás para frente do alfabeto em df2
df1 = pd.DataFrame({
    'nome': ['Alice', 'Bob', 'Carol', 'David', 'Eve'],
    'fruta': ['Apple', 'Banana', 'Cherry', 'Date', 'Elderberry']
})

df2 = pd.DataFrame({
    'nome_z': ['Zoe', 'Yanni', 'Xena', 'Walter', 'Violet'],
    'fruta_z': ['Zucchini', 'Yam', 'Ximenia', 'Watermelon', 'Voavanga']
})

# Realizando o cross join entre df1 e df2
cross_joined_df = df1.merge(df2, how='cross')
cross_joined_df


In [ ]:

temp_cvm_web = run.sys_load_pkl(f'{b3.app_folder}/temp_cvm_web')


In [ ]:
temp_cvm_web = run.sys_save_pkl(temp_cvm_web, f'{b3.app_folder}/cvm')


In [ ]:
b3_cvm = run.sys_load_pkl(f'{b3.app_folder}/b3_cvm')


In [ ]:
b3_cvm.keys()

In [ ]:
mask = b3_cvm['BENS INDUSTRIAIS']['CNPJ_CIA'] == '00.924.429/0001-75'
mask &= b3_cvm['BENS INDUSTRIAIS']['CD_CONTA'] == '3.01'
b3_cvm['BENS INDUSTRIAIS'][mask]['VL_CONTA'].plot()

In [4]:
cvm_local = run.sys_load_pkl(f'{b3.app_folder}/cvm')
cvm_web = run.sys_load_pkl(f'{b3.app_folder}/temp_cvm_web')


In [6]:
def cvm_get_updated_rows(df_local, df_web, df_columns, year):

    # Define the list of columns that should not be included as key columns for merging.
    no_columns = ['VL_CONTA']

    # Generate the list of key columns for merging by excluding the columns listed in no_columns.
    key_columns = [col for col in df_columns if col not in no_columns]

    # Merge the local and web DataFrames based on the key columns, and an indicator column 
    df_merged = pd.merge(df_local, df_web, on=key_columns, how='outer', suffixes=('_local', '_web'), indicator=True)

    # Define conditions to determine which values to keep in the merged DataFrame.
    conditions = [
        df_merged['_merge'] == 'left_only',  # Rows that exist only in the local DataFrame.
        df_merged['_merge'] == 'right_only',  # Rows that exist only in the web DataFrame.
        (df_merged['_merge'] == 'both') & (df_merged['VL_CONTA_local'] != df_merged['VL_CONTA_web'])  # Rows that exist in both but have different 'VL_CONTA' values.
    ]

    # Define the choices corresponding to each condition.
    choices = [
        df_merged['VL_CONTA_local'],  # Keep the local value for 'VL_CONTA' if the row is from local only.
        df_merged['VL_CONTA_web'],    # Keep the web value for 'VL_CONTA' if the row is from web only.
        df_merged['VL_CONTA_web']     # Keep the web value for 'VL_CONTA' if the values differ between local and web.
    ]

    # Apply the conditions to create a new 'VL_CONTA' column in the merged DataFrame.
    df_merged['VL_CONTA'] = np.select(conditions, choices, default=df_merged['VL_CONTA_web'])

    # Create a mask to identify rows that have been updated or are new from the web DataFrame.
    updated_rows_mask = (
        (df_merged['_merge'] == 'right_only') |
        ((df_merged['_merge'] == 'both') & (df_merged['VL_CONTA_local'] != df_merged['VL_CONTA_web']))
    )

    # Use the mask to filter the merged DataFrame to only include updated or new rows.
    updated_rows = df_merged[updated_rows_mask]

    # Define the columns to be used for the final filtering.
    filt_cols = ['CNPJ_CIA', 'AGRUPAMENTO', 'CD_CONTA', 'DS_CONTA']

    # Perform an inner merge to filter the original merged DataFrame using the updated rows 
    # and keep only the rows with matching values in the specified filter columns.
    cvm_web = pd.merge(
        updated_rows[filt_cols],  # Only the columns to match from the updated rows.
        df_merged,  # The original merged DataFrame.
        on=filt_cols,  # Columns to match on.
        how='inner'  # Keep only matches found in both DataFrames.
    )[df_columns]

    # Display the number of updated rows.
    print(f'{year} {len(updated_rows)}/{len(df_merged)} linhas foram atualizadas')

    return cvm_web


In [7]:
def cvm_updated_rows(cvm_local, cvm_web):
    """
    Merge two dictionaries of dataframes and extract updated rows.

    The function performs an outer merge on two dictionaries of dataframes, `cvm_local` (existing data) 
    and `cvm_web` (new data). The purpose is to update old financial data with new financial data and 
    to identify rows which have been updated for future mathematical transformations.

    Parameters:
    - cvm_local (dict): Dictionary with years as keys and existing financial data as values (pandas DataFrames).
    - cvm_web (dict): Dictionary with years as keys and new financial data as values (pandas DataFrames).

    Returns:
    - tuple: Two dictionaries of dataframes - the first contains the merged data, and the second contains the updated rows.

    """
    
    # Yearly updated data.
    cvm_updated = {}
    # Compile a list of years present in either local or web data.
    years = sorted(set(cvm_local.keys()).union(cvm_web.keys()))

    # Get column names from the earliest year's data.
    df_columns = (
        cvm_local.get(min(years), pd.DataFrame()).columns
        if min(years) in cvm_local
        else cvm_web[min(years)].columns
    )

    # Process each year's data.
    for year in years:
        # Obtain local and web data for the year.
        df_local = cvm_local.get(year, pd.DataFrame(columns=df_columns))
        df_web = cvm_web.get(year, pd.DataFrame(columns=df_columns))

        # Update rows by comparing local and web data.
        cvm_updated[year] = cvm_get_updated_rows(df_local, df_web, df_columns, year)

    return cvm_updated


In [14]:
cvm_web = cvm_updated_rows(cvm_local, cvm_web)



2020 3/2168387 linhas foram atualizadas


In [7]:
df = cvm_web[2020]
df[df['BALANCE_SHEET'] == 'DRE'].head(10)

,FILENAME,DEMONSTRATIVO,BALANCE_SHEET,ANO,AGRUPAMENTO,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC,COLUNA_DF
2553349,itr_cia_aberta_DRE_con_2020.csv,itr,DRE,2020,con,00.001.180/0001-26,2020-03-31,1,CENTRAIS ELET BRAS SA ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,2020-03-31,3.01,Receita de Venda de Bens e/ou Serviços,6955636.0,S,2020-01-01,NaN
2553351,itr_cia_aberta_DRE_con_2020.csv,itr,DRE,2020,con,00.001.180/0001-26,2020-03-31,1,CENTRAIS ELET BRAS SA ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,2020-03-31,3.02,Custo dos Bens e/ou Serviços Vendidos,-1750995.0,S,2020-01-01,NaN
2553353,itr_cia_aberta_DRE_con_2020.csv,itr,DRE,2020,con,00.001.180/0001-26,2020-03-31,1,CENTRAIS ELET BRAS SA ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,2020-03-31,3.02.01,Energia comprada para revenda,-646220.0,N,2020-01-01,NaN
2553355,itr_cia_aberta_DRE_con_2020.csv,itr,DRE,2020,con,00.001.180/0001-26,2020-03-31,1,CENTRAIS ELET BRAS SA ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,2020-03-31,3.02.02,Encargos sobre o uso da rede elétrica,-446459.0,N,2020-01-01,NaN
2553357,itr_cia_aberta_DRE_con_2020.csv,itr,DRE,2020,con,00.001.180/0001-26,2020-03-31,1,CENTRAIS ELET BRAS SA ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,2020-03-31,3.02.03,Combustível para produção de energia elétrica,-467998.0,N,2020-01-01,NaN
2553359,itr_cia_aberta_DRE_con_2020.csv,itr,DRE,2020,con,00.001.180/0001-26,2020-03-31,1,CENTRAIS ELET BRAS SA ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,2020-03-31,3.02.04,Construção,-190318.0,N,2020-01-01,NaN
2553361,itr_cia_aberta_DRE_con_2020.csv,itr,DRE,2020,con,00.001.180/0001-26,2020-03-31,1,CENTRAIS ELET BRAS SA ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,2020-03-31,3.03,Resultado Bruto,5204641.0,S,2020-01-01,NaN
2553363,itr_cia_aberta_DRE_con_2020.csv,itr,DRE,2020,con,00.001.180/0001-26,2020-03-31,1,CENTRAIS ELET BRAS SA ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,2020-03-31,3.04,Despesas/Receitas Operacionais,-2871222.0,S,2020-01-01,NaN
2553365,itr_cia_aberta_DRE_con_2020.csv,itr,DRE,2020,con,00.001.180/0001-26,2020-03-31,1,CENTRAIS ELET BRAS SA ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,2020-03-31,3.04.01,Despesas com Vendas,0.0,S,2020-01-01,NaN
2553367,itr_cia_aberta_DRE_con_2020.csv,itr,DRE,2020,con,00.001.180/0001-26,2020-03-31,1,CENTRAIS ELET BRAS SA ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,2020-03-31,3.04.02,Despesas Gerais e Administrativas,-3060487.0,S,2020-01-01,NaN


In [38]:
filename = 'acoes'
columns = ['Companhia', 'Trimestre', 'Ações ON', 'Ações PN', 'Ações ON em Tesouraria', 'Ações PN em Tesouraria', 'URL']
acoes = run.sys_read_or_create_dataframe(filename, columns)
acoes.to_csv('acoes.csv')
acoes

acoes: total 34 items


,Companhia,Trimestre,Ações ON,Ações PN,Ações ON em Tesouraria,Ações PN em Tesouraria,URL
0,BANCO INDUSTRIAL DO BRASIL,31122010,111895475,56649630,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
1,TOTVS SA,31122010,31459,0,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
2,BCO BRADESCO SA,31122010,1881225,1881225,395,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
3,REDECARD SA,31122010,672971,0,210448,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
4,BCO SANTANDER BRASIL SA,31122010,212841732,186202385,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
5,NUMERAL 80 PARTICIPACOES SA,31122010,452567,203209,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
6,SANTOS BRASIL PARTICIPACOES SA,31122010,452567,203209,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
7,ENGIE BRASIL ENERGIA SA,31122010,652742192,0,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
8,TARPON INVESTIMENTOS SA,31122010,41207,0,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
9,ROMI SA,31122010,74758,0,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...


In [30]:
filename = 'nsd_links'
cols_nsd = ['company', 'dri', 'dri2', 'dre', 'data', 'versao', 'auditor', 'auditor_rt', 'cancelamento', 'protocolo', 'envio', 'url', 'nsd']

nsd = run.sys_read_or_create_dataframe(filename, cols_nsd)
nsd.to_csv('nsd_links.csv')

nsd_links: total 119375 items


In [29]:
mask = nsd['company'] == 'BCO BRASIL SA'
mask &= nsd['dre'].isin(['INFORMACOES TRIMESTRAIS', 'DEMONSTRACOES FINANCEIRAS PADRONIZADAS'])
nsd[mask]

,company,dri,dri2,dre,data,versao,auditor,auditor_rt,cancelamento,protocolo,envio,url,nsd
0,BCO BRASIL SA,IVAN DE SOUZA MONTEIRO,FCA V3,DEMONSTRACOES FINANCEIRAS PADRONIZADAS,31122010,V1,KPMG AUDITORES INDEPENDENTES,FRANCESCO LUIGI CELSO,,001023DFP31122010010000482272,2011-02-17 18:50:38,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,4822
0,BCO BRASIL SA,IVAN DE SOUZA MONTEIRO,FCA V3,DEMONSTRACOES FINANCEIRAS PADRONIZADAS,31122010,V2,KPMG AUDITORES INDEPENDENTES,FRANCESCO LUIGI CELSO,AJUSTE NA FORMATACAO DAS DECLARACOES DOS DIRET...,001023DFP31122010020000482371,2011-02-17 19:34:12,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,4823
0,BCO BRASIL SA,IVAN DE SOUZA MONTEIRO,FCA V2,DEMONSTRACOES FINANCEIRAS PADRONIZADAS,31122010,V3,KPMG AUDITORES INDEPENDENTES,FRANCESCO LUIGI CELSO,INCLUSAO DAS DEMONSTRACOES CONTABEIS CONSOLIDA...,001023DFP31122010030000694577,2011-04-30 03:29:48,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,6945
0,BCO BRASIL SA,IVAN DE SOUZA MONTEIRO,FCA V2,INFORMACOES TRIMESTRAIS,31032011,V1,KPMG AUDITORES INDEPENDENTES,GIUSEPPE MASI,,001023ITR31032011010000743970,2011-05-10 09:50:14,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,7439
0,BCO BRASIL SA,IVAN DE SOUZA MONTEIRO,FCA V2,INFORMACOES TRIMESTRAIS,30062011,V1,KPMG AUDITORES INDEPENDENTES,GIUSEPPE MASI,,001023ITR30062011010001061682,2011-08-09 09:26:27,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,10616
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,BCO BRASIL SA,DANIEL ALVES MARIA,FCA V1,INFORMACOES TRIMESTRAIS,30062022,V1,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENTES,LUIZ CARLOS OSELIERO FILHO,,001023ITR30062022010011944578,2022-08-10 18:52:56,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,119445
0,BCO BRASIL SA,DANIEL ALVES MARIA,FCA V1,INFORMACOES TRIMESTRAIS,30092022,V1,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENTES,LUIZ CARLOS OSELIERO FILHO,,001023ITR30092022010012157374,2022-11-09 19:24:45,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,121573
0,BCO BRASIL SA,DANIEL ALVES MARIA,FCA V1,DEMONSTRACOES FINANCEIRAS PADRONIZADAS,31122022,V1,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENTES,LUIZ CARLOS OSELIERO FILHO,,001023DFP31122022010012349867,2023-02-13 18:55:51,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,123498
0,BCO BRASIL SA,MARCO GEOVANNE TOBIAS DA SILVA,FCA V2,INFORMACOES TRIMESTRAIS,31032023,V1,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENTES,LUIZ CARLOS OSELIERO FILHO,,001023ITR31032023010012675679,2023-05-15 20:08:22,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,126756


In [36]:
# Sorting the dataframe
nsd['nsd'] = nsd['nsd'].astype(int)
nsd_data_sorted = nsd.sort_values(by=['nsd'], ascending=[True])

# Dropping duplicates, keeping the first occurrence (which is the one with the highest 'nsd' due to the sorting)
nsd_data_most_recent = nsd_data_sorted.drop_duplicates(subset=['company', 'data'], keep='last')

# Checking the result
mask = nsd_data_most_recent['company'] == 'BCO BRASIL SA'
mask &= nsd_data_most_recent['dre'].isin(['INFORMACOES TRIMESTRAIS', 'DEMONSTRACOES FINANCEIRAS PADRONIZADAS'])
nsd_data_most_recent[mask].sort_values(by=['nsd'], ascending=[True])

,company,dri,dri2,dre,data,versao,auditor,auditor_rt,cancelamento,protocolo,envio,url,nsd
0,BCO BRASIL SA,IVAN DE SOUZA MONTEIRO,FCA V2,DEMONSTRACOES FINANCEIRAS PADRONIZADAS,31122010,V3,KPMG AUDITORES INDEPENDENTES,FRANCESCO LUIGI CELSO,INCLUSAO DAS DEMONSTRACOES CONTABEIS CONSOLIDA...,001023DFP31122010030000694577,2011-04-30 03:29:48,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,6945
0,BCO BRASIL SA,IVAN DE SOUZA MONTEIRO,FCA V2,INFORMACOES TRIMESTRAIS,31032011,V1,KPMG AUDITORES INDEPENDENTES,GIUSEPPE MASI,,001023ITR31032011010000743970,2011-05-10 09:50:14,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,7439
0,BCO BRASIL SA,IVAN DE SOUZA MONTEIRO,FCA V2,INFORMACOES TRIMESTRAIS,30062011,V1,KPMG AUDITORES INDEPENDENTES,GIUSEPPE MASI,,001023ITR30062011010001061682,2011-08-09 09:26:27,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,10616
0,BCO BRASIL SA,IVAN DE SOUZA MONTEIRO,FCA V2,INFORMACOES TRIMESTRAIS,30092011,V1,KPMG AUDITORES INDEPENDENTES,GIUSEPPE MASI,,001023ITR30092011010001230671,2011-11-03 08:59:27,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,12306
0,BCO BRASIL SA,IVAN DE SOUZA MONTEIRO,FCA V2,DEMONSTRACOES FINANCEIRAS PADRONIZADAS,31122011,V4,KPMG AUDITORES INDEPENDENTES,GIUSEPPE MASI,INCLUSAO DE VALORES NO QUADRO CONCILIACAO DO P...,001023DFP31122011040001604576,2012-04-09 11:47:09,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,16045
0,BCO BRASIL SA,IVAN DE SOUZA MONTEIRO,FCA V2,INFORMACOES TRIMESTRAIS,31032012,V1,KPMG AUDITORES INDEPENDENTES,GIUSEPPE MASI,,001023ITR31032012010001667671,2012-05-03 10:08:04,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,16676
0,BCO BRASIL SA,IVAN DE SOUZA MONTEIRO,FCA V2,INFORMACOES TRIMESTRAIS,30062012,V2,KPMG AUDITORES INDEPENDENTES,GIUSEPPE MASI,SUBSTITUICAO DO RELATORIO ESPECIAL SEM RESSALVA,001023ITR30062012020002047262,2012-08-14 12:34:59,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,20472
0,BCO BRASIL SA,IVAN DE SOUZA MONTEIRO,FCA V2,INFORMACOES TRIMESTRAIS,30092012,V1,KPMG AUDITORES INDEPENDENTES,GIUSEPPE MASI,,001023ITR30092012010002221178,2012-11-08 10:06:31,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,22211
0,BCO BRASIL SA,IVAN DE SOUZA MONTEIRO,FCA V2,DEMONSTRACOES FINANCEIRAS PADRONIZADAS,31122012,V4,KPMG AUDITORES INDEPENDENTES,GIUSEPPE MASI,AJUSTE REDACIONAL NO PARECER DOS AUDITORES IND...,001023DFP31122012040002525765,2013-03-25 20:27:30,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,25257
0,BCO BRASIL SA,IVAN DE SOUZA MONTEIRO,FCA V2,INFORMACOES TRIMESTRAIS,31032013,V1,KPMG AUDITORES INDEPENDENTES,GIUSEPPE MASI,,001023ITR31032013010002746578,2013-05-15 09:06:47,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,27465


In [187]:
acoes_df = acoes
nsd_links_df = nsd

acoes_df.reset_index(drop=True, inplace=True)
nsd_links_df.reset_index(drop=True, inplace=True)

In [177]:
nsd_links_df = nsd_links_df.drop(index=4968)


In [178]:
# Aplicamos o filtro prévio para selecionar apenas as linhas de interesse em nsd_links
selected_dre = ['INFORMACOES TRIMESTRAIS', 'DEMONSTRACOES FINANCEIRAS PADRONIZADAS']
filtered_nsd = nsd_links_df[nsd_links_df['dre'].isin(selected_dre)].copy()

# Renomeação das colunas para alinhar com 'acoes'
filtered_nsd.rename(columns={'company': 'Companhia', 'data': 'Trimestre'}, inplace=True)

# Conversão da coluna 'nsd' para int e ordenação dos dados para garantir que o mais recente esteja no topo
filtered_nsd['nsd'] = filtered_nsd['nsd'].astype(int)

filtered_nsd.sort_values(by='nsd', ascending=True, inplace=True)

# Ajuste do formato de data
filtered_nsd['Trimestre'] = pd.to_datetime(filtered_nsd['Trimestre'], errors='coerce', format='%d%m%Y')

mask = filtered_nsd['Companhia'] == 'BCO ABC BRASIL SA'
mask &= filtered_nsd['Trimestre'] == '2010-12-31'
filtered_nsd[mask]

,Companhia,dri,dri2,dre,Trimestre,versao,auditor,auditor_rt,cancelamento,protocolo,envio,url,nsd
3042,BCO ABC BRASIL SA,SERGIO LULIA JACOB,FCA V3,DEMONSTRACOES FINANCEIRAS PADRONIZADAS,2010-12-31,V1,ERNST YOUNG AUDITORES INDEPENDENTES SS,EDUARDO WELLICHEN,,020958DFP31122010010000471273,2011-02-11 09:47:39,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,4712
3043,BCO ABC BRASIL SA,SERGIO LULIA JACOB,FCA V3,DEMONSTRACOES FINANCEIRAS PADRONIZADAS,2010-12-31,V2,ERNST YOUNG AUDITORES INDEPENDENTES SS,EDUARDO WELLICHEN,CORRECAO NA ESCALA DE R PARA R MIL,020958DFP31122010020000471372,2011-02-11 10:17:46,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,4713


In [179]:
# Remoção de duplicatas, mantendo apenas a entrada mais recente para cada par company-data
filtered_nsd.drop_duplicates(subset=['Companhia', 'Trimestre'], keep='last', inplace=True)

mask = filtered_nsd['Companhia'] == 'BCO ABC BRASIL SA'
mask &= filtered_nsd['Trimestre'] == '2010-12-31'
filtered_nsd[mask]

,Companhia,dri,dri2,dre,Trimestre,versao,auditor,auditor_rt,cancelamento,protocolo,envio,url,nsd
3043,BCO ABC BRASIL SA,SERGIO LULIA JACOB,FCA V3,DEMONSTRACOES FINANCEIRAS PADRONIZADAS,2010-12-31,V2,ERNST YOUNG AUDITORES INDEPENDENTES SS,EDUARDO WELLICHEN,CORRECAO NA ESCALA DE R PARA R MIL,020958DFP31122010020000471372,2011-02-11 10:17:46,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,4713


In [181]:
# Extração do 'nsd' da URL em 'acoes'
acoes_df['nsd'] = acoes_df['URL'].apply(lambda x: int(x.split('Documento=')[-1].split('&')[0]))

# Conversão da coluna 'Trimestre' para datetime em 'acoes' para alinhar com 'filtered_nsd'
acoes_df['Trimestre'] = pd.to_datetime(acoes_df['Trimestre'], errors='coerce', format='%d%m%Y')

mask = acoes_df['Companhia'] == 'BCO ABC BRASIL SA'
mask &= acoes_df['Trimestre'] == '2010-12-31'
acoes_df[mask]

,Companhia,Trimestre,Ações ON,Ações PN,Ações ON em Tesouraria,Ações PN em Tesouraria,URL,nsd
12,BCO ABC BRASIL SA,2010-12-31,67822120,67822120,0,1192801,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,4712
13,BCO ABC BRASIL SA,2010-12-31,67822120,67822120,0,1192801,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,4713


In [182]:
# Encontrar o nsd mais alto para cada grupo Companhia-Trimestre em filtered_nsd
nsd_max_filtered_nsd = filtered_nsd.groupby(['Companhia', 'Trimestre'])['nsd'].max().reset_index()


mask = nsd_max_filtered_nsd['Companhia'] == 'BCO ABC BRASIL SA'
mask &= nsd_max_filtered_nsd['Trimestre'] == '2010-12-31'
nsd_max_filtered_nsd[mask]

,Companhia,Trimestre,nsd
3143,BCO ABC BRASIL SA,2010-12-31,4713


In [183]:
# Merge entre acoes e nsd_max_filtered_nsd para identificar quais linhas devem ser mantidas em acoes
acoes_updated = pd.merge(acoes_df, nsd_max_filtered_nsd, on=['Companhia', 'Trimestre', 'nsd'], how='left', indicator=True)

mask = acoes_updated['Companhia'] == 'BCO ABC BRASIL SA'
mask &= acoes_updated['Trimestre'] == '2010-12-31'
acoes_updated[mask]

,Companhia,Trimestre,Ações ON,Ações PN,Ações ON em Tesouraria,Ações PN em Tesouraria,URL,nsd,_merge
12,BCO ABC BRASIL SA,2010-12-31,67822120,67822120,0,1192801,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,4712,left_only
13,BCO ABC BRASIL SA,2010-12-31,67822120,67822120,0,1192801,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,4713,both


In [184]:
# Merge entre acoes e nsd_max_filtered_nsd para comparar os nsd
acoes_updated = pd.merge(acoes_df, nsd_max_filtered_nsd, on=['Companhia', 'Trimestre'], how='left', suffixes=('', '_max'))

# Removemos as linhas de acoes onde o nsd é menor que o nsd máximo para a respectiva 'Companhia' e 'Trimestre'
acoes_most_recent = acoes_updated[acoes_updated['nsd'] >= acoes_updated['nsd_max']]

# Removendo as colunas auxiliares usadas para o merge
acoes_most_recent = acoes_most_recent.drop(columns=['nsd_max'])

mask = acoes_most_recent['Companhia'] == 'BCO ABC BRASIL SA'
mask &= acoes_most_recent['Trimestre'] == '2010-12-31'
acoes_most_recent[mask]

,Companhia,Trimestre,Ações ON,Ações PN,Ações ON em Tesouraria,Ações PN em Tesouraria,URL,nsd
13,BCO ABC BRASIL SA,2010-12-31,67822120,67822120,0,1192801,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,4713


In [185]:
# Primeiro, obtemos uma lista de URLs presentes no DataFrame acoes_most_recent
urls_to_remove = acoes_most_recent['URL'].unique()

# Agora, filtramos o DataFrame filtered_nsd para remover as linhas cuja URL está na lista urls_to_remove
filtered_nsd = filtered_nsd[~filtered_nsd['url'].isin(urls_to_remove)]

mask = filtered_nsd['Companhia'] == 'BCO ABC BRASIL SA'
mask &= filtered_nsd['Trimestre'] == '2010-12-31'
filtered_nsd[mask]

,Companhia,dri,dri2,dre,Trimestre,versao,auditor,auditor_rt,cancelamento,protocolo,envio,url,nsd


In [202]:
def synchronize_nsd_and_acoes(nsd, acoes):
    """
    Update and synchronize the nsd and acoes dataframes.

    This function processes the nsd dataframe to include only the rows with specific 'dre' types,
    indicating they are either quarterly reports or standardized financial statements.
    It then renames columns to match those in acoes, converts the 'nsd' column to integers, and
    sorts the data to put the most recent documents at the top. After filtering out duplicates,
    the function extracts the 'nsd' from the URL in the acoes dataframe and merges the two dataframes.
    It removes any rows in acoes that have an 'nsd' less than the maximum for their respective company
    and quarter. Finally, it filters out any URLs from filtered_nsd that are present in acoes,
    ensuring no overlap between the two dataframes in terms of URLs.

    Parameters:
    nsd (DataFrame): The nsd dataframe to be processed.
    acoes (DataFrame): The acoes dataframe to be updated based on nsd.

    Returns:
    DataFrame: The updated filtered_nsd dataframe with duplicates and outdated entries removed.
    DataFrame: The updated acoes dataframe with only the most recent entries for each company and quarter.
    """
    # Filtrar nsd para incluir apenas as linhas que têm 'dre' como 'INFORMACOES TRIMESTRAIS' ou
    # 'DEMONSTRACOES FINANCEIRAS PADRONIZADAS'. Isso é como se estivéssemos selecionando
    # apenas os documentos que são relatórios trimestrais ou demonstrações financeiras padronizadas.
    selected_dre = ['INFORMACOES TRIMESTRAIS', 'DEMONSTRACOES FINANCEIRAS PADRONIZADAS']
    filtered_nsd = nsd[nsd['dre'].isin(selected_dre)].copy()

    # Renomear colunas para que 'company' se torne 'Companhia' e 'data' se torne 'Trimestre'.
    # Isso é necessário para que as colunas correspondam às de acoes, permitindo futuras comparações e combinações.
    filtered_nsd.rename(columns={'company': 'Companhia', 'data': 'Trimestre'}, inplace=True)

    # Converter a coluna 'nsd' para inteiros para permitir comparações numéricas e
    # ordenar as linhas para que os documentos mais recentes fiquem no topo.
    filtered_nsd['nsd'] = filtered_nsd['nsd'].astype(int)
    filtered_nsd.sort_values(by='nsd', ascending=True, inplace=True)

    # Converter a coluna 'Trimestre' para o tipo datetime para permitir comparações de datas.
    # Isso transforma strings que representam datas em objetos de data reais.
    filtered_nsd['Trimestre'] = pd.to_datetime(filtered_nsd['Trimestre'], errors='coerce', format='%d%m%Y')

    # Remover duplicatas em filtered_nsd, mantendo apenas a entrada mais recente para cada combinação de Companhia-Trimestre.
    # Isso garante que, se tivermos múltiplos documentos para a mesma empresa e trimestre, só o mais recente seja mantido.
    filtered_nsd.drop_duplicates(subset=['Companhia', 'Trimestre'], keep='last', inplace=True)

    # Extrair o número sequencial do documento (nsd) da URL no DataFrame acoes.
    # Isso é feito dividindo a URL em partes e pegando o número que aparece logo após 'Documento='.
    acoes['nsd'] = acoes['URL'].apply(lambda x: int(x.split('Documento=')[-1].split('&')[0]))

    # Converter a coluna 'Trimestre' em acoes para o tipo datetime, assim como fizemos com filtered_nsd.
    acoes['Trimestre'] = pd.to_datetime(acoes['Trimestre'], errors='coerce', format='%d%m%Y')

    # Agrupar filtered_nsd por Companhia e Trimestre e pegar o maior nsd para cada grupo.
    # Isso nos dá o documento mais recente para cada empresa e trimestre.
    nsd_max_filtered_nsd = filtered_nsd.groupby(['Companhia', 'Trimestre'])['nsd'].max().reset_index()

    # Fazer um merge (junção) entre acoes e nsd_max_filtered_nsd para combinar as linhas com base em Companhia e Trimestre.
    # Adicionamos um indicador para ver se a linha resultante é uma correspondência ('both') ou não ('left_only').
    # Isso ajuda a identificar quais linhas em acoes têm um documento correspondente em nsd que é o mais recente.
    acoes_updated = pd.merge(acoes, nsd_max_filtered_nsd, on=['Companhia', 'Trimestre', 'nsd'], how='left', indicator=True)

    # Realizar outra junção entre acoes e nsd_max_filtered_nsd para comparar os nsd.
    # Desta vez, a junção é feita para trazer o nsd máximo ao lado do nsd atual em acoes.
    acoes_updated = pd.merge(acoes, nsd_max_filtered_nsd, on=['Companhia', 'Trimestre'], how='left', suffixes=('', '_max'))

    # Remover as linhas de acoes onde o nsd é menor que o nsd máximo para a respectiva Companhia e Trimestre.
    # Isso efetivamente atualiza acoes para manter apenas as entradas mais recentes.
    acoes_most_recent = acoes_updated[acoes_updated['nsd'] >= acoes_updated['nsd_max']].drop(columns=['nsd_max'])

    # Obter uma lista de URLs presentes em acoes_most_recent para saber quais URLs devemos remover de filtered_nsd.
    urls_to_remove = acoes_most_recent['URL'].unique()

    # Filtrar filtered_nsd para remover as linhas cuja URL está na lista de urls_to_remove.
    # Isso assegura que não haja sobreposição entre acoes_most_recent e filtered_nsd em termos de URLs.
    filtered_nsd = filtered_nsd[~filtered_nsd['url'].isin(urls_to_remove)]

    # Revert the column names in filtered_nsd to their original state
    filtered_nsd.rename(columns={'Companhia': 'company', 'Trimestre': 'data'}, inplace=True)
    
    # Drop the 'nsd' and 'nsd_max' columns from acoes_updated
    acoes_updated.drop(columns=['nsd', 'nsd_max'], inplace=True)
    
    return filtered_nsd, acoes_updated


# Example usage of the function
filename = 'acoes'
columns = ['Companhia', 'Trimestre', 'Ações ON', 'Ações PN', 'Ações ON em Tesouraria', 'Ações PN em Tesouraria', 'URL']
acoes = run.sys_read_or_create_dataframe(filename, columns)
filename = 'nsd_links'
cols_nsd = ['company', 'dri', 'dri2', 'dre', 'data', 'versao', 'auditor', 'auditor_rt', 'cancelamento', 'protocolo', 'envio', 'url', 'nsd']
nsd = run.sys_read_or_create_dataframe(filename, cols_nsd)

filtered_nsd, acoes = synchronize_nsd_and_acoes(nsd, acoes)


acoes: total 34 items
nsd_links: total 119375 items


In [203]:
acoes

,Companhia,Trimestre,Ações ON,Ações PN,Ações ON em Tesouraria,Ações PN em Tesouraria,URL
0,BANCO INDUSTRIAL DO BRASIL,2010-12-31,111895475,56649630,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
1,TOTVS SA,2010-12-31,31459,0,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
2,BCO BRADESCO SA,2010-12-31,1881225,1881225,395,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
3,REDECARD SA,2010-12-31,672971,0,210448,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
4,BCO SANTANDER BRASIL SA,2010-12-31,212841732,186202385,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
5,NUMERAL 80 PARTICIPACOES SA,2010-12-31,452567,203209,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
6,SANTOS BRASIL PARTICIPACOES SA,2010-12-31,452567,203209,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
7,ENGIE BRASIL ENERGIA SA,2010-12-31,652742192,0,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
8,TARPON INVESTIMENTOS SA,2010-12-31,41207,0,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
9,ROMI SA,2010-12-31,74758,0,0,0,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...


In [205]:
acoes = run.sys_load_pkl('acoes')
nsd_max_filtered_nsd = run.sys_load_pkl('nsd_max_filtered_nsd')


In [208]:
acoes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 30
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Companhia               31 non-null     object        
 1   Trimestre               31 non-null     datetime64[ns]
 2   Ações ON                31 non-null     object        
 3   Ações PN                31 non-null     object        
 4   Ações ON em Tesouraria  31 non-null     object        
 5   Ações PN em Tesouraria  31 non-null     object        
 6   URL                     31 non-null     object        
 7   nsd                     31 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 2.2+ KB


In [207]:
nsd_max_filtered_nsd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43618 entries, 0 to 43617
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Companhia  43618 non-null  object        
 1   Trimestre  34042 non-null  datetime64[ns]
 2   nsd        43618 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 1022.4+ KB
